# t|ket> benchmarking example


Warning: This notebook is now deprecated for versions from 0.2.0 onwards, and is not going to be updated in ongoing releases.

The aim of this example is to show how to run the IBM benchmarking circuits through t|ket>. You will need both `pytket` and `pytket_qiskit` installed from pip before running this turoial. You will also need `pandas` to capture the data.

The benchmarking circuits originated from https://github.com/iic-jku/ibm_qx_mapping/tree/master/examples, but there is a copy in pytket in the "benchmarking" folder. The initial circuits are written in QASM, meaning that they must be converted to t|ket>'s internal representation via Qiskit. Using this script we will compile these circuits through t|ket> and then print a table to terminal containing analysis of the circuits post-compilation.

First, begin by importing qiskit and pytket.

In [1]:
import qiskit
import pytket

We need access to Qiskit's `QuantumCircuit` and the `circuit_to_dag` converter, which will allow us to take the original QASM and retrieve a Qiskit DAGCircuit. There is then a pytket method `dagcircuit_to_tk` to convert this DAGCircuit to t|ket>'s internal representation. We also need the pytket method `coupling_to_arc` to convert IBM's architectures to the t|ket> `Architecture` class for the use of routing.

In [2]:
from qiskit import QuantumCircuit

from pytket.qiskit import qiskit_to_tk

We need all of the required pytket equipment to allow us to perform clean-up transform passes, to route and to analyse the circuits. Lastly, we require the `pandas` module to hold our data, and 
`time` to benchmark compilation time per circuit.

In [3]:
from pytket import Architecture, route
from pytket import OpType, Transform
import pandas
import time

The architectures used to benchmark for our routing paper were the IBMqx5 and IBMTokyo architectures. The architectures qx5 and Tokyo respectively are shown below (although the diagrams do not show the directedness of the coupling maps). These diagrams are from https://www.research.ibm.com/ibm-q/technology/devices/#ibmqx5. We will now define the coupling map representing both architectures. These will later be converted to directed graphs to be used by t|ket>.
![alt text](IBMqx5Arc.png "Title")
![alt text](IBMTokyoArc.png "Title2")

In [4]:
ibm_devices = {
    "ibmqx5": {"edges": [(1, 0), (1, 2), (2, 3), (3, 4), 
    (3, 14), (5, 4), (6, 5), (6, 7), (6, 11), (7, 10), 
    (8, 7), (9, 8), (9, 10), (11, 10), (12, 5), (12, 11), 
    (12, 13), (13, 4), (13, 14), (15, 0), (15, 2), (15, 14)],
    "nodes": 16},
    "ibmq_20_tokyo": {"edges": [(0, 1), (0, 5), (1, 2), (1, 6), (1, 7), (2, 3), (2, 6),
    (2, 7), (3, 4), (3, 8), (3, 9), (4, 8), (4, 9), (5, 6), (5, 10), (5, 11), (6, 7),
    (6, 10), (6, 11), (7, 8), (7, 12), (7, 13), (8, 9), (8, 12), (8, 13), (9, 14), (10, 11),
    (10, 15), (11, 12), (11, 16), (11, 17), (12, 13), (12, 16), (12, 17), (13, 14), (13, 18),
    (13, 19), (14, 18), (14, 19), (15, 16), (16, 17), (17, 18), (18, 19)], "nodes": 20}
}

We shall now choose the device and create a directed graph for t|ket>'s routing to use.

In [5]:
device_name = 'ibmqx5' #'ibm_20_tokyo' ###Note: can also be ran using the Tokyo machine architecture
                                       ###      or with a user-defined coupling map
coupling_map = ibm_devices[device_name]["edges"]
directed_arc = Architecture(coupling_map)

We now define a method which takes in (1) a QASM file and (2) a directed graph architecture; it returns analysis of the circuit after our transform passes and routing procedure have been completed. It will also print out to terminal the time taken for all the transformations and routing to finish. Changing which optimisations are run in this method will trade off quality of the ouputs for time taken. For example, removing all optimisation passes and just running the routing procedure will give the fastest run time but may leave some redundant gates in the final circuit.

In [6]:
def getStats(filename, directed_arc):
    qc = QuantumCircuit.from_qasm_file(filename)
    tkcirc = qiskit_to_tk(qc)
    start_time = time.process_time()
    Transform.OptimisePhaseGadgets().apply(tkcirc)
    outcirc = route(tkcirc, directed_arc)
    # decompose swaps to CX gates and redirect CXs in wrong direction
    outcirc.decompose_SWAP_to_CX()
    outcirc.redirect_CX_gates(directed_arc)
    Transform.OptimisePostRouting().apply(outcirc)
    
    time_elapsed = time.process_time() - start_time
    
    print("Compilation time for circuit " + str(filename) + ": " + str(time_elapsed) + "s")
    if outcirc.n_gates==0:
        return [0,0,0,0,0]
    ###Returns: [number of vertices, circuit depth, nubmer of CX gates, number of parallel slices of CX gates]
    return [outcirc.n_gates, outcirc.depth(), outcirc.n_gates_of_type(OpType.CX), 
            outcirc.depth_by_type(OpType.CX), time_elapsed]
    ###Note: the raw number of vertices in the circuits and the raw depth 
    ###      need to have the i/o vertices removed for fair comparisons

Lastly, we generate the table of input QASM filenames from a csv file using `pandas` and run the circuits through our compiler. The results are printed to terminal by default.

In [7]:
test_table = pandas.read_csv("benchmarking/IBMQConfig.csv",index_col=0)
test_table = test_table.sort_values(by='Depth in')

stat_table = pandas.DataFrame({})
total_time = 0

for i, (index, row) in enumerate(test_table.iterrows()):
    filename = row['Filename']
    new_stats = getStats(filename, directed_arc)
    total_time += new_stats[4] ###
    print("Time spent compiling so far: " + str(total_time))
    new_table_row = pandas.DataFrame.from_dict({index : new_stats}, 
    orient='index', columns=['Size out', 'Depth out', 'CX count out', 'CX depth out','Runtime'])
    stat_table = stat_table.append(new_table_row)

#stat_table.to_csv("BenchmarkTket.csv") ###Note: uncomment this line to print table to csv.
with pandas.option_context('display.max_rows', None):
    print(stat_table)

Compilation time for circuit benchmarking/ibmq/xor5_254.qasm: 0.00951000000000013s
Time spent compiling so far: 0.00951000000000013
Compilation time for circuit benchmarking/ibmq/graycode6_47.qasm: 0.005398000000000014s
Time spent compiling so far: 0.014908000000000143
Compilation time for circuit benchmarking/ibmq/ex1_226.qasm: 0.008242000000000083s
Time spent compiling so far: 0.023150000000000226
Compilation time for circuit benchmarking/ibmq/4gt11_84.qasm: 0.015388999999999875s
Time spent compiling so far: 0.0385390000000001
Compilation time for circuit benchmarking/ibmq/4mod5-v0_20.qasm: 0.020800999999999625s
Time spent compiling so far: 0.059339999999999726
Compilation time for circuit benchmarking/ibmq/ex-1_166.qasm: 0.017431999999999892s
Time spent compiling so far: 0.07677199999999962
Compilation time for circuit benchmarking/ibmq/4mod5-v1_22.qasm: 0.01641899999999996s
Time spent compiling so far: 0.09319099999999958
Compilation time for circuit benchmarking/ibmq/mod5d1_63.qas

Compilation time for circuit benchmarking/ibmq/4mod7-v1_96.qasm: 0.1598600000000001s
Time spent compiling so far: 5.620319000000002
Compilation time for circuit benchmarking/ibmq/4gt4-v0_80.qasm: 0.18252500000000182s
Time spent compiling so far: 5.802844000000004
Compilation time for circuit benchmarking/ibmq/mod10_176.qasm: 0.17993299999999834s
Time spent compiling so far: 5.982777000000002
Compilation time for circuit benchmarking/ibmq/0410184_169.qasm: 0.6244690000000013s
Time spent compiling so far: 6.6072460000000035
Compilation time for circuit benchmarking/ibmq/qft_16.qasm: 0.05671399999999949s
Time spent compiling so far: 6.6072460000000035
Compilation time for circuit benchmarking/ibmq/4gt12-v0_88.qasm: 0.20898000000000394s
Time spent compiling so far: 6.8162260000000074
Compilation time for circuit benchmarking/ibmq/rd84_142.qasm: 0.7401959999999974s
Time spent compiling so far: 7.556422000000005
Compilation time for circuit benchmarking/ibmq/rd53_311.qasm: 0.5026390000000021

Time spent compiling so far: 1036.8230970000002
Compilation time for circuit benchmarking/ibmq/sqn_258.qasm: 28.884308000000146s
Time spent compiling so far: 1065.7074050000003
Compilation time for circuit benchmarking/ibmq/inc_237.qasm: 106.20392800000013s
Time spent compiling so far: 1171.9113330000005
Compilation time for circuit benchmarking/ibmq/cm85a_209.qasm: 89.76953500000013s
Time spent compiling so far: 1261.6808680000006
Compilation time for circuit benchmarking/ibmq/rd84_253.qasm: 66.61587499999996s
Time spent compiling so far: 1328.2967430000006
Compilation time for circuit benchmarking/ibmq/co14_215.qasm: 128.39122300000008s
Time spent compiling so far: 1456.6879660000006
Compilation time for circuit benchmarking/ibmq/root_255.qasm: 123.82956399999989s
Time spent compiling so far: 1580.5175300000005
Compilation time for circuit benchmarking/ibmq/mlp4_245.qasm: 242.6016039999995s
Time spent compiling so far: 1823.119134
Compilation time for circuit benchmarking/ibmq/urf2_2